In [13]:
import numpy as np
from  scipy import sparse
import time

#store matrix G in a compressed sparse matrix format.  P = (D)^-1 * G
#Note that rows contain the "to" nodes and the columns contain "from" nodes
def data_to_csr():
    row = []
    col = [] 
    row_col = []
    num_of_edges =0
    nodes = set([])
    weight_dict = {}
    with open("stanweb.dat",'r') as f:
        for line in f.readlines():
            lst = line.split()
            node1, node2, weights = int(lst[0])-1,int(lst[1])-1,float(lst[2])
            row.append(node1) #storing "to" nodes on rows of G
            col.append(node2) #storing "from" nodes on columns of G
            num_of_edges += 1 
            nodes.add(node1)
            nodes.add(node2)
            weight_dict[node1] = weights #creating outdegrees dictionary
        
        #inserting nodes with outdegree 0 on the out degree dictionary
        for node in nodes:
            if node not in weight_dict.keys():
                weight_dict[node] = [0]
                
    return sparse.csr_matrix(([1]*num_of_edges,(col,row)),shape=(len(nodes),len(nodes))), weight_dict

#constructing dictionary where keys are the nodes2 of the file and values are the nodes that contain links to the key
def construct_giving_list():
    nodes=set([])
    out_list = {}
    with open("stanweb.dat",'r') as f:
        for line in f.readlines():
            lst = line.split()
            node1, node2, weights = int(lst[0])-1,int(lst[1])-1, float(lst[2])
            nodes.add(node1)
            nodes.add(node2)
            try:
                out_list[node2].append(node1)
            except:
                out_list[node2]=[node1]
                
        for node in nodes:
            if node not in out_list.keys():
                out_list[node]=[]
    return out_list

In [7]:
#Create matrices that will be used below
G, out_degree = data_to_csr() #G is sparse compressed matrix, outdegree is a dictionary
n = G.shape[0] #number of nodes
giving_pages = construct_giving_list() 

##### Page Rank Implementation 

In the following function I implement the pagerank algorithm according to Google's algorithm using the formula:

x(k).T=x(k-1).T*P + (1-α)*v.T 
 
 I manipulated the implementation of the power method of the paper as it was impossible to create (1-α)*v.T due to memory error. So I first calculate a* x(k-1) *P  excluding dangling nodes and then add (1-a)/n to all the nodes including dangling ones to avoid storing vector v.

In [14]:
#The algorithm is based on the formula rank = alpha *ranks/out_degree + (1−alpha)/n
def PageRank(G,n, a=0.85, t = 10**-8):
    
    d = G.sum(axis=0).T #calculate number of outlinks in each node-webpage
    
    #Initialization
    fast_conv = []
    ranks = np.ones((n,1))/n #vector
    iterations = 0
    error = 1
    
    #Iterating till convergence
    while error > t:       
        #try except is used to ignore division with 0 as some outdegrees are zero
        try:
            new_ranks = G.dot(a*(ranks/ d))  #G.dot(1/d) is the P hat matrix.
        except (ZeroDivisionError): 
            pass  
        
        #Add the second part of the formula used to correct dangling nodes
        new_ranks += (1-a)/n
    
        #Check which nodes converge on iteration 1
        iterations +=1
        if iterations<2:
            for i in range(len(ranks)):
                if np.linalg.norm(ranks[i]-new_ranks[i])< t:
                    fast_conv.append([i])
                    
        #Stop condition
        error = np.linalg.norm(ranks-new_ranks)/np.linalg.norm(ranks)  
        ranks = new_ranks
    return new_ranks, iterations,fast_conv

##### Gauss Seidel Implementation

In [15]:
#This implementation is based on the paper http://www.w3c.ethz.ch/CDstore/www2002/poster/173.pdf on the formula on page 3
def Gauss_Seidel(G,outdegree,a,n,threshold = 10**-8):
    rank_prev = np.ones(n)/n
    iterations = 0
    fast_conv=[]
    err = 1
    rank = rank_prev.copy()
    flag = False
   
    while flag == False:
        iterations +=1        
        for i in range(n):
            summation1 = 0
            summation2 = 0
            for j in G[i]:
                if j < i:                                          
                    summation1 += a *outdegree[j] * rank[j]
                if j > i:                                          
                    summation2 +=  a * outdegree[j] * rank_prev[j]
            rank[i] = (1-a) + summation1 + summation2

        err = np.linalg.norm(rank - rank_prev)/np.linalg.norm(rank)
        if err < threshold:
            flag = True
        
        
        if iterations<2:
            for i in range(len(rank)):
                if np.linalg.norm(rank[i]- rank_prev[i])< threshold:
                    fast_conv.append([i])
        rank_prev = rank.copy()
        
    return rank,iterations,fast_conv

##### Question a:  Are the results the same for both methods? Which method seems to be faster? 

##### Calculate Pagerank using Gauss Seidel for a=0.85

In [18]:

#linking_pages, in_degree_values = construct_indegree_list()
start = time.time()
results_gs,iterations,_ = Gauss_Seidel(giving_pages,out_degree ,0.85,n,10**-8)
end = time.time()
print("Time to execute:" ,(end-start)/60, "minutes")
indices_gs_85 = results_gs.argsort()[-n:][::-1]
print("\nThe 20 max ranked nodes are:")
print(indices_gs_85[:50])
print("\nNumber of iterations:", iterations)

Time to execute: 2.8148815274238586 minutes

The 20 max ranked nodes are:
[ 89072 226410 241453 262859 134831 234703 136820  68888 105606  69357
  67755 225871 186749 272441 251795  95162 119478 231362  55787 167294
 179644  38341 117151 198089  60209 235495 132694 181700 259454 247240
 120707  62477 161889 221086 183003 176789 137631  77998  17780  96744
 112741 145891 151427  81434  60439 208541     90 214127 258347 222872]

Number of iterations: 57


##### Calculate Pagerank using Power Method for a=0.85

In [16]:
start =  time.time()
ranks,iterations, _ = PageRank(G,n, a=0.85, t = 10**-8)
end = time.time()
#Get indices of the max ranked
ranks = np.asarray(ranks).ravel()
indices_pr_85 = ranks.argsort()[-n:][::-1]
print("Time to execute:",end-start,"seconds")
print("\nThe 20 max ranked nodes are:")
print(indices_pr_85[:50])
print("\nNumber of iterations:", iterations)

C:\Users\nataz\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()


Time to execute: 4.942476511001587 seconds

The 20 max ranked nodes are:
[ 89072 226410 241453 262859 134831 234703 136820  68888 105606  69357
  67755 225871 186749 251795 272441  95162 119478 231362  55787 167294
 179644  38341 117151 198089  60209 235495 132694 181700 247240 259454
 120707  62477 161889  17780 176789 221086  77998 183003 137631  96744
 112741 145891 151427  81434  60439 208541     90 214127 258347 222872]

Number of iterations: 95


In [21]:
#Compare results
flag = True
counter=0
k = 5 #check if the ranking of a node using gauss seidel is the same with the rank of that node +-5 positions using Power Method
for i in range(len(indices_gs_85)-k) :
    if indices_pr_85[i] not in indices_gs_85[i-k : i+k]:
        flag = False
        counter += 1
if flag==True:
    print("The results are the same for both methods")
else:
    print("The number of nodes ranked significantly differently(more than 5 positions difference) are:", counter)

The number of nodes ranked significantly differently(more than 5 positions difference) are: 57977


Answer: 
The results are the not exactly the same for Pagerank and Gauss Seidel but while observing the 50 most important results we can refer that the rankings differ for only one to 5 positions for most nodes.

Speed: 
Pagerank is significantly faster as it is possible to use dot products. But it is important to note that in bibliography it is mentioned that Gauss Seidel can be faster in some types of matrices.

##### Question b :Do the previous task with α = 0.99.Your remarks on the convergence speed. Did the ranking of the ﬁrst 50 nodes changed?

##### Run Gauss Seidel

In [22]:
#linking_pages, in_degree_values = construct_indegree_list()
start = time.time()
results_gs,_,_ = Gauss_Seidel(giving_pages,out_degree ,0.99,n,10**-8)
end = time.time()
print("Time to execute:" ,(end-start)/60, "minutes")
indices_gs = results_gs.argsort()[-n:][::-1]
print("\nThe 20 max ranked nodes are:")
print(indices_gs[:20])

Time to execute: 24.674164497852324 minutes

The 20 max ranked nodes are:
[ 89072 281771 174664 226410 179644 271408 262859 136820  68888  77987
 116529 272441 251795  95162  65579 119478 241453 245764  58047  14784]


##### Calculate using Power Method

In [24]:
#Calculate using Power Method
start =  time.time()
ranks,iterations, _ = PageRank(G,n, a=0.99, t = 10**-8)
end = time.time()

#Get indices of the max ranked
ranks = np.asarray(ranks).ravel()
indices_b = ranks.argsort()[-n:][::-1]

print("Time to execute:",end-start,"seconds")
print("\nThe 20 max ranked nodes are:")
print(indices_b[:20])
print("\nNumber of iterations:", iterations)


C:\Users\nataz\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()


Time to execute: 68.7914092540741 seconds

The 20 max ranked nodes are:
[ 89072 281771 174664 226410 179644 271408 262859 136820  68888  77987
 116529 272441  95162 251795  65579 119478 241453 245764  58047  14784]

Number of iterations: 1514


##### Compare top 50 nodes of Power method with a=0.85 to the top 50 of the power method with a = 0.99

In [25]:
flag = True
counter=0
for i in range(50) :
    if indices_b[i] != indices_pr_85[i]:
        flag = False
        counter += 1
if flag==True:
    print("The results are the same for both methods")
else:
    print("The number of top 50 ranked pages significantly differently are:", counter)

The number of top 50 ranked pages significantly differently are: 48


#####  Compare top 50 nodes of Gauss Seidel with a=0.85 to the top 50 of the Gauss Seidel with a = 0.99

In [26]:
flag = True
counter=0
for i in range(50) :
    if indices_gs[i] != indices_gs_85[i]:
        flag = False
        counter += 1
if flag==True:
    print("The results are the same for both methods")
else:
    print("The number of top 50 ranked pages significantly differently are:", counter)

The number of top 50 ranked pages significantly differently are: 49


Power method for a=0.88 runs in 5 seconds while the one with a=0.99 runs in approximatelly 40 seconds. 
Gauss Seidel with a=0.85 converges in 3 minutes while gauss seidel with a=0.99 converges in 45 minutes.

This delay of convergence happens because 1-a which is the probability of teleportation to another non-connected node is too small and thus the surfer stays trapped into spider traps and deadlocks for many iterations.  
 

##### Question C : When we use the power method do all the cmponents of π converge at the same speed to their limits? If not which of the converge faster: those that correspond to important nodes or to non important ? Do you observe the same behavior when you ﬁnd π through the solution of the linear system?

##### Power Method 

In [27]:
#Power Method
ranks,iterations, conv_results = PageRank(G,n, a=0.85, t = 10**-8)

#Get indices of the max ranked
ranks = np.asarray(ranks).ravel()
indices = ranks.argsort()[-G.shape[0]:][::-1]

print("The  number of nodes that had already converged from the first iteration is:", len(conv_results))

#Check if any of the first 100 highly ranked nodes are in the list of the nodes that converged in the first iteration
flag = False
for ind in indices[:100]:
    if ind in conv_results:
        print("The important nodes(top 100) that converged on iteration 1 are:",ind)
        flag = True

if flag == False:
    print("\nNot one of the 100 highly ranked nodes converged on the first iteration.")

C:\Users\nataz\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()


The  number of nodes that had already converged from the first iteration is: 14542

Not one of the 100 highly ranked nodes converged on the first iteration.


##### Gauss Seidel 

In [95]:
ranks,iterations, conv_results = Gauss_Seidel(giving_pages,out_degree ,0.85,n,10**-8)

#Get indices of the max ranked
ranks = np.asarray(ranks).ravel()
indices = ranks.argsort()[-G.shape[0]:][::-1]

print("The  number of nodes that had already converged from the first iteration is:", len(conv_results))

#Check if any of the first 100 highly ranked nodes are in the list of the nodes that converged in the first iteration
flag = False
for ind in indices[:100]:
    if ind in conv_results:
        print("The important nodes(top 100) that converged on iteration 1 are:",ind)
        flag = True

if flag == False:
    print("\nNot one of the 100 highly ranked nodes converged on the first iteration.")

The  number of nodes that had already converged from the first iteration is: 0

Not one of the 100 highly ranked nodes converged on the first iteration.
